# TP : eQTL et Randomisation Mendélienne
## Master ST4Health — Génétique Humaine

<div style="background-color:#e8f4f8; padding:12px; border-radius:6px;">

**Objectifs du TP :**
1. Comprendre le concept d'eQTL et son rôle de variable instrumentale
2. Explorer des données de summary statistics GWAS
3. Réaliser une analyse de Randomisation Mendélienne (MR) manuellement
4. Utiliser le package `MendelianRandomization` et interpréter les analyses de sensibilité

**Durée : ~1h40** (après l'introduction magistrale)

</div>

---

> **Note sur les données** : Les données de ce TP sont synthétiques mais réalistes.  
> Les données eQTL sont simulées à partir des valeurs publiées dans GTEx v8 (tissu foie, gène *HMGCR*).  
> Les IVs MR sont basés sur de vrais rsIDs et des tailles d'effet tirées de la littérature  
> (Willer et al. *Nat Genet* 2013 pour le LDL ; Nikpay et al. *Nat Genet* 2015 pour la CAD).  
> Le code pour reproduire les données depuis les sources originales est fourni en annexe.

## 0. Setup

Chargement de `rpy2` et installation des packages R si nécessaire.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
# Installation des packages si nécessaire (à ne lancer qu'une fois)
if (!require("MendelianRandomization", quietly=TRUE)) {
    install.packages("MendelianRandomization", repos="https://cloud.r-project.org")
}
if (!require("ggplot2", quietly=TRUE)) install.packages("ggplot2", repos="https://cloud.r-project.org")
if (!require("dplyr",   quietly=TRUE)) install.packages("dplyr",   repos="https://cloud.r-project.org")
cat("Packages OK\n")

---
## Partie I — eQTL comme variable instrumentale (~20 min)

### Contexte biologique

La **randomisation mendélienne** repose sur l'idée que les variants génétiques sont aléatoirement distribués dans la population (comme dans un essai randomisé). Pour qu'un SNP soit un bon **instrument génétique**, il doit satisfaire trois hypothèses :

1. **Relevance** : le SNP est associé à l'exposition (ici : il affecte l'expression d'un gène)
2. **Indépendance** : le SNP est indépendant des facteurs confondants
3. **Exclusion restriction** : le SNP n'affecte l'outcome que *via* l'exposition

Un **eQTL** (*expression Quantitative Trait Locus*) est exactement ça : un SNP dont on sait qu'il régule l'expression d'un gène. Il satisfait naturellement l'hypothèse 1.

### Notre exemple : *HMGCR* et le cholestérol LDL

Le gène *HMGCR* code pour la HMG-CoA réductase, enzyme clé de la voie de biosynthèse du cholestérol — et **cible des statines**. Un cis-eQTL dans le foie sur *HMGCR* peut donc servir d'instrument pour estimer l'effet causal du LDL sur les maladies cardiovasculaires.

Le SNP **rs12916** (chr5:74,656,528, build GRCh37) est le lead cis-eQTL de *HMGCR* dans le foie (GTEx v8).

In [ ]:
%%R
# Chargement des données eQTL
eqtl <- read.csv("eqtl_HMGCR.csv")
cat("Dimensions :", nrow(eqtl), "individus x", ncol(eqtl), "variables\n")
head(eqtl)

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1a** — Décrivez la distribution des génotypes pour rs12916.  
Combien d'individus ont 0, 1 ou 2 copies de l'allèle T ?  
Comparez à la fréquence allélique attendue sous Hardy-Weinberg (MAF = 0.42).

</div>

In [ ]:
%%R
# Votre réponse ici
table(eqtl$genotype_rs12916)

# Fréquences observées vs attendues
n <- nrow(eqtl)
maf <- 0.42
cat("\nFréquences HWE attendues :\n")
cat("  0 copies :", round((1-maf)^2 * n), "\n")
cat("  1 copie  :", round(2*maf*(1-maf) * n), "\n")
cat("  2 copies :", round(maf^2 * n), "\n")

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1b** — Visualisez la relation entre le génotype rs12916 et l'expression de *HMGCR*.  
Quel est le sens de l'effet ? Est-ce cohérent avec la biologie ?

</div>

In [ ]:
%%R
library(ggplot2)

# Boxplot expression ~ génotype
eqtl$genotype_factor <- factor(eqtl$genotype_rs12916,
                                levels=0:2,
                                labels=c("CC (0T)", "CT (1T)", "TT (2T)"))

ggplot(eqtl, aes(x=genotype_factor, y=HMGCR_expression, fill=genotype_factor)) +
    geom_boxplot(alpha=0.7, outlier.shape=16) +
    geom_jitter(width=0.1, alpha=0.3, size=0.8) +
    scale_fill_manual(values=c("#4575b4","#91bfdb","#e0f3f8")) +
    labs(
        title="cis-eQTL de HMGCR dans le foie",
        subtitle="SNP rs12916 (chr5:74,656,528)",
        x="Génotype rs12916",
        y="Expression HMGCR (log2-normalized)",
        fill="Génotype"
    ) +
    theme_minimal(base_size=13) +
    theme(legend.position="none")

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1c** — Testez statistiquement l'association eQTL par régression linéaire.  
Ajustez sur l'âge, le sexe et PC1.  
Quelle est la taille d'effet (beta) ? Est-ce significatif ?

</div>

In [ ]:
%%R
# Modèle 1 : eQTL brut (sans covariables)
mod_brut <- lm(HMGCR_expression ~ genotype_rs12916, data=eqtl)
cat("=== Modèle brut ===\n")
summary(mod_brut)$coefficients

# Modèle 2 : ajusté sur covariables
mod_adj <- lm(HMGCR_expression ~ genotype_rs12916 + age + sex + PC1, data=eqtl)
cat("\n=== Modèle ajusté (age, sex, PC1) ===\n")
summary(mod_adj)$coefficients

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1d** — Calculez la **statistique F** de l'instrument :

$$F = \frac{\hat{\beta}^2}{SE^2}$$

Une règle empirique : F > 10 indique un instrument fort. Qu'en est-il ici ?  
Interprétez : qu'est-ce qu'un instrument *faible* biaiserait dans l'analyse MR ?

</div>

In [ ]:
%%R
beta_eqtl <- coef(summary(mod_adj))["genotype_rs12916", "Estimate"]
se_eqtl   <- coef(summary(mod_adj))["genotype_rs12916", "Std. Error"]

F_stat <- (beta_eqtl / se_eqtl)^2
cat(sprintf("Beta eQTL : %.4f\n", beta_eqtl))
cat(sprintf("SE        : %.4f\n", se_eqtl))
cat(sprintf("F-stat    : %.1f\n", F_stat))
cat(sprintf("\n=> Instrument %s (F %s 10)\n",
    ifelse(F_stat > 10, "FORT", "faible"),
    ifelse(F_stat > 10, ">", "<")))

---
## Partie II — Exploration des données MR (~15 min)

### Passage à l'échelle : du gène au GWAS

L'approche eQTL sur un gène est illustrative, mais une MR robuste utilise **de nombreux instruments génétiques** tirés d'un GWAS sur l'exposition.

Ici l'exposition est le **cholestérol LDL** et l'outcome est la **maladie coronarienne (CAD)**.

Le fichier `MR_IVs_LDL_CAD.csv` contient 77 SNPs instrumentaux déjà sélectionnés et harmonisés :
- Associés au LDL à *p* < 5×10⁻⁸ (seuil GWAS)
- Indépendants (clumpés, r² < 0.01, fenêtre 10 Mb)
- Effets alignés sur l'allèle *augmentant* le LDL
- Présents dans les deux GWAS (LDL et CAD)

In [ ]:
%%R
# Chargement des données MR
mr_data <- read.csv("MR_IVs_LDL_CAD.csv")
cat("Dimensions :", nrow(mr_data), "SNPs x", ncol(mr_data), "variables\n\n")
head(mr_data[, c("SNP","GENE","BETA_LDL","SE_LDL","P_LDL","BETA_CAD","SE_CAD","F_stat")])

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 2a** — Quelle est la distribution des statistiques F dans ce jeu de données ?  
Tous les instruments sont-ils forts ?  
Notez les gènes connus (HMGCR, PCSK9, LDLR, APOE) : sont-ils présents ?

</div>

In [ ]:
%%R
cat("F-stat : min =", round(min(mr_data$F_stat),1),
    " médiane =", round(median(mr_data$F_stat),1),
    " max =", round(max(mr_data$F_stat),1), "\n")
cat("Proportion F > 10 :", mean(mr_data$F_stat > 10), "\n\n")

# Gènes biologiquement importants
genes_cles <- c("HMGCR","PCSK9","LDLR","APOE","APOB","CETP","LPA")
mr_data[mr_data$GENE %in% genes_cles,
        c("SNP","GENE","BETA_LDL","BETA_CAD","F_stat")]

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 2b** — Faites un scatter plot beta_CAD ~ beta_LDL.  
Que remarquez-vous ? Quelle relation attendez-vous si LDL *cause* la CAD ?

</div>

In [ ]:
%%R
ggplot(mr_data, aes(x=BETA_LDL, y=BETA_CAD, label=ifelse(F_stat > 500, GENE, ""))) +
    geom_point(aes(size=F_stat), alpha=0.6, color="#2166ac") +
    geom_smooth(method="lm", formula=y~x-1, se=TRUE,
                color="#d73027", linetype="dashed", linewidth=1) +
    geom_hline(yintercept=0, linetype="dotted", color="grey50") +
    geom_vline(xintercept=0, linetype="dotted", color="grey50") +
    geom_text(hjust=-0.15, vjust=0.5, size=3, color="#555555") +
    labs(
        title="Effets SNP sur LDL-C vs effets SNP sur CAD",
        subtitle="Pente = estimé MR (IVW) — chaque point est un instrument génétique",
        x=expression(hat(beta)[LDL]~"(effet sur le LDL-C, log scale)"),
        y=expression(hat(beta)[CAD]~"(effet sur la CAD, log OR)"),
        size="F-stat"
    ) +
    theme_minimal(base_size=13)

---
## Partie III — MR à la main : IVW (~25 min)

### Principe : du Wald ratio à l'IVW

Pour chaque SNP *j*, le **ratio de Wald** estime l'effet causal de l'exposition sur l'outcome :

$$\hat{\theta}_j = \frac{\hat{\beta}_{Y,j}}{\hat{\beta}_{X,j}}$$

Son erreur standard approximée est :

$$SE(\hat{\theta}_j) \approx \frac{SE_{Y,j}}{|\hat{\beta}_{X,j}|}$$

La méthode **Inverse Variance Weighted (IVW)** combine ces estimés en une moyenne pondérée par l'inverse de la variance :

$$\hat{\theta}_{IVW} = \frac{\sum_j w_j \hat{\theta}_j}{\sum_j w_j} \quad \text{avec} \quad w_j = \frac{1}{SE(\hat{\theta}_j)^2}$$

**Équivalence clé :** l'IVW est strictement équivalent à une régression linéaire pondérée de $\hat{\beta}_Y$ sur $\hat{\beta}_X$ sans intercept, avec poids $1/SE_Y^2$.

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3a** — Calculez le ratio de Wald et son SE pour chaque SNP.  
Quel SNP a le ratio le plus élevé ? le plus faible ?

</div>

In [ ]:
%%R
# Ratio de Wald par SNP
mr_data$wald_ratio <- mr_data$BETA_CAD / mr_data$BETA_LDL
mr_data$wald_se    <- mr_data$SE_CAD   / abs(mr_data$BETA_LDL)

cat("Wald ratio le plus élevé :\n")
print(mr_data[which.max(mr_data$wald_ratio), c("SNP","GENE","wald_ratio","wald_se")])
cat("\nWald ratio le plus faible :\n")
print(mr_data[which.min(mr_data$wald_ratio), c("SNP","GENE","wald_ratio","wald_se")])

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3b** — Construisez un **forest plot** des ratios de Wald.  
Que révèle l'hétérogénéité entre les instruments ?

</div>

In [ ]:
%%R
library(dplyr)

# Ordonner par wald_ratio pour le forest plot
fd <- mr_data %>%
    arrange(wald_ratio) %>%
    mutate(
        ci_lo = wald_ratio - 1.96 * wald_se,
        ci_hi = wald_ratio + 1.96 * wald_se,
        SNP_label = paste0(SNP, " (", GENE, ")")
    )
fd$SNP_label <- factor(fd$SNP_label, levels=fd$SNP_label)

ggplot(fd, aes(x=wald_ratio, y=SNP_label)) +
    geom_point(color="#2166ac", size=1.5) +
    geom_errorbarh(aes(xmin=ci_lo, xmax=ci_hi), height=0, color="#2166ac", alpha=0.5) +
    geom_vline(xintercept=0, linetype="dashed", color="grey40") +
    labs(
        title="Forest plot — Ratios de Wald par instrument",
        x="Wald ratio (effet causal estimé LDL → CAD)",
        y=NULL
    ) +
    theme_minimal(base_size=9) +
    theme(axis.text.y=element_text(size=7))

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3c** — Calculez l'estimé IVW **manuellement** (moyenne pondérée des ratios de Wald).  
Calculez le SE, l'IC 95% et la p-value.  
Interprétez : est-ce que le LDL cause la CAD ?

</div>

In [ ]:
%%R
# IVW = moyenne pondérée des ratios de Wald
weights     <- 1 / mr_data$wald_se^2
ivw_est     <- sum(weights * mr_data$wald_ratio) / sum(weights)
ivw_se      <- sqrt(1 / sum(weights))
ivw_z       <- ivw_est / ivw_se
ivw_pval    <- 2 * (1 - pnorm(abs(ivw_z)))
ivw_ci_lo   <- ivw_est - 1.96 * ivw_se
ivw_ci_hi   <- ivw_est + 1.96 * ivw_se

cat("=== IVW (moyenne pondérée) ===\n")
cat(sprintf("Estimé  : %.4f\n", ivw_est))
cat(sprintf("SE      : %.4f\n", ivw_se))
cat(sprintf("IC 95%%  : [%.4f ; %.4f]\n", ivw_ci_lo, ivw_ci_hi))
cat(sprintf("p-value : %.2e\n", ivw_pval))
cat(sprintf("\n=> Pour 1 unité d'augmentation du LDL (log scale),\n"))
cat(sprintf("   l'odds ratio de CAD est : %.3f (IC: %.3f - %.3f)\n",
    exp(ivw_est), exp(ivw_ci_lo), exp(ivw_ci_hi)))

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3d** — Vérifiez votre résultat par **régression WLS sans intercept**.  
Les deux méthodes donnent-elles exactement le même résultat ? Pourquoi ?

</div>

In [ ]:
%%R
# IVW = régression WLS de beta_CAD ~ beta_LDL, sans intercept, poids = 1/SE_CAD^2
mod_ivw <- lm(BETA_CAD ~ -1 + BETA_LDL,
              weights = 1/SE_CAD^2,
              data = mr_data)

ivw_reg_est <- coef(mod_ivw)["BETA_LDL"]
ivw_reg_se  <- sqrt(vcov(mod_ivw)["BETA_LDL","BETA_LDL"])

cat("=== IVW (régression WLS) ===\n")
cat(sprintf("Estimé  : %.4f\n", ivw_reg_est))
cat(sprintf("SE      : %.4f\n", ivw_reg_se))
cat(sprintf("\nDifférence avec méthode précédente : %.6f\n",
    abs(ivw_est - ivw_reg_est)))
cat("(Les deux méthodes sont strictement équivalentes)\n")

---
## Partie IV — Analyses de sensibilité à la main (~30 min)

### Pourquoi des analyses de sensibilité ?

L'IVW suppose que **tous les instruments sont valides** (pas de pleiotropie directe).  
En pratique, certains SNPs peuvent affecter l'outcome par des voies indépendantes de l'exposition : c'est la **pleiotropie horizontale**.

Trois méthodes complémentaires permettent de tester la robustesse :

| Méthode | Hypothèse clé | Formule |
|---|---|---|
| **IVW** | Tous les IVs valides | Moyenne pondérée des Wald ratios |
| **MR-Egger** | Pleiotropie proportionnelle à l'effet (InSIDE) | Régression *avec* intercept |
| **Weighted Median** | ≥ 50% des IVs valides | Médiane pondérée des Wald ratios |

On va implémenter les trois à la main.

### IV.1 — Récapitulatif des ratios de Wald

On repart du `merged_df` calculé en Partie III.

In [ ]:
%%R
# Rappel : les données sont déjà dans mr_data depuis la Partie III
# avec les colonnes wald_ratio et wald_se calculées
cat("Nombre d'instruments :", nrow(mr_data), "\n")
head(mr_data[, c("SNP","GENE","BETA_LDL","BETA_CAD","wald_ratio","wald_se")])

---
### IV.2 — MR-Egger

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Principe :** MR-Egger est une régression WLS de $\hat{\beta}_Y$ sur $\hat{\beta}_X$ **avec intercept** :

$$\hat{\beta}_Y = \alpha + \theta \cdot \hat{\beta}_X + \epsilon$$

- La **pente** $\theta$ est l'estimé causal (corrigé de la pleiotropie moyenne)
- L'**intercept** $\alpha$ est un test de pleiotropie directionnelle : si $\alpha \neq 0$, il y a pleiotropie

**Attention :** MR-Egger suppose que les $\hat{\beta}_X$ sont tous positifs (allèles alignés sur l'effet augmentant l'exposition). On a déjà fait cette harmonisation dans nos données.

</div>

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px; margin-top:8px;">

**Question 4a** — Implémentez MR-Egger par régression WLS avec intercept.  
Comparez la pente à l'estimé IVW de la Partie III.  
L'intercept est-il significativement différent de 0 ?

</div>

In [ ]:
%%R
# MR-Egger : régression WLS avec intercept
# Poids = 1 / SE_CAD^2
mod_egger <- lm(BETA_CAD ~ BETA_LDL,
                weights = 1 / SE_CAD^2,
                data    = mr_data)

egger_slope     <- coef(mod_egger)["BETA_LDL"]
egger_intercept <- coef(mod_egger)["(Intercept)"]
egger_se_slope  <- sqrt(vcov(mod_egger)["BETA_LDL", "BETA_LDL"])
egger_se_int    <- sqrt(vcov(mod_egger)["(Intercept)", "(Intercept)"])

# p-values (test bilatéral)
egger_p_slope <- 2 * (1 - pnorm(abs(egger_slope   / egger_se_slope)))
egger_p_int   <- 2 * (1 - pnorm(abs(egger_intercept / egger_se_int)))

cat("=== MR-Egger ===\n")
cat(sprintf("Pente (effet causal) : %.4f  SE: %.4f  p: %.2e\n",
            egger_slope, egger_se_slope, egger_p_slope))
cat(sprintf("Intercept (pleiotropie) : %.4f  SE: %.4f  p: %.3f\n",
            egger_intercept, egger_se_int, egger_p_int))
cat(sprintf("\n=> L'intercept %s significativement != 0 (p %s 0.05)\n",
    ifelse(egger_p_int < 0.05, "EST", "n'est PAS"),
    ifelse(egger_p_int < 0.05, "<", ">=")))

---
### IV.3 — Weighted Median

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Principe :** la médiane pondérée des ratios de Wald est robuste si **au moins 50%** du poids total vient d'instruments valides. C'est plus robuste que l'IVW face aux outliers.

L'algorithme :
1. Trier les ratios de Wald par valeur croissante
2. Calculer les poids cumulés normalisés
3. Trouver la valeur correspondant au quantile 0.5

Le SE s'obtient par **bootstrap**.

</div>

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px; margin-top:8px;">

**Question 4b** — Implémentez le Weighted Median.  
Comparez à l'IVW et à MR-Egger.

</div>

In [ ]:
%%R
# Weighted Median
weighted_median <- function(wald_ratios, weights) {
    # Trier par ratio de Wald
    ord       <- order(wald_ratios)
    ratios_s  <- wald_ratios[ord]
    weights_s <- weights[ord]
    
    # Poids cumulés normalisés
    w_norm <- weights_s / sum(weights_s)
    w_cum  <- cumsum(w_norm)
    
    # Interpolation au quantile 0.5
    below <- which(w_cum < 0.5)
    if (length(below) == 0) return(ratios_s[1])
    idx <- max(below)
    
    # Interpolation linéaire entre idx et idx+1
    w_low  <- w_cum[idx]
    w_high <- w_cum[idx + 1]
    r_low  <- ratios_s[idx]
    r_high <- ratios_s[idx + 1]
    
    r_low + (0.5 - w_low) / (w_high - w_low) * (r_high - r_low)
}

# Bootstrap pour le SE
bootstrap_wm_se <- function(wald_ratios, weights, n_boot=1000) {
    n <- length(wald_ratios)
    boot_estimates <- replicate(n_boot, {
        idx <- sample(n, replace=TRUE)
        weighted_median(wald_ratios[idx], weights[idx])
    })
    sd(boot_estimates)
}

# Application
set.seed(42)
weights_wm <- 1 / mr_data$wald_se^2

wm_est <- weighted_median(mr_data$wald_ratio, weights_wm)
wm_se  <- bootstrap_wm_se(mr_data$wald_ratio, weights_wm, n_boot=1000)
wm_z   <- wm_est / wm_se
wm_p   <- 2 * (1 - pnorm(abs(wm_z)))

cat("=== Weighted Median ===\n")
cat(sprintf("Estimé  : %.4f\n", wm_est))
cat(sprintf("SE      : %.4f\n", wm_se))
cat(sprintf("IC 95%%  : [%.4f ; %.4f]\n", wm_est - 1.96*wm_se, wm_est + 1.96*wm_se))
cat(sprintf("p-value : %.2e\n", wm_p))

---
### IV.4 — Comparaison des méthodes

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4c** — Assemblez les résultats des trois méthodes dans un tableau et un forest plot.  
Les estimés sont-ils cohérents ? Que conclure ?

</div>

In [ ]:
%%R
# Tableau comparatif
# IVW depuis Partie III
weights_ivw <- 1 / mr_data$wald_se^2
ivw_est     <- sum(weights_ivw * mr_data$wald_ratio) / sum(weights_ivw)
ivw_se      <- sqrt(1 / sum(weights_ivw))

results <- data.frame(
    Methode  = c("IVW", "MR-Egger", "Weighted Median"),
    Estimate = c(ivw_est,      egger_slope,     wm_est),
    SE       = c(ivw_se,       egger_se_slope,  wm_se),
    stringsAsFactors = FALSE
)
results$CI_lo  <- results$Estimate - 1.96 * results$SE
results$CI_hi  <- results$Estimate + 1.96 * results$SE
results$pvalue <- 2 * (1 - pnorm(abs(results$Estimate / results$SE)))
results$OR     <- round(exp(results$Estimate), 3)

print(results[, c("Methode","Estimate","SE","CI_lo","CI_hi","pvalue","OR")])

In [ ]:
%%R
library(ggplot2)

# Forest plot des 3 méthodes
results$Methode <- factor(results$Methode,
                           levels=c("Weighted Median","MR-Egger","IVW"))

ggplot(results, aes(x=Estimate, y=Methode, color=Methode)) +
    geom_point(size=4) +
    geom_errorbarh(aes(xmin=CI_lo, xmax=CI_hi), height=0.15, linewidth=1) +
    geom_vline(xintercept=0, linetype="dashed", color="grey40") +
    scale_color_manual(values=c("IVW"="#2166ac",
                                 "MR-Egger"="#d73027",
                                 "Weighted Median"="#1a9850")) +
    labs(
        title    = "Comparaison des méthodes MR — LDL-C → CAD",
        subtitle = "Estimés (log OR) avec IC 95%",
        x        = "Estimé causal (log OR par unité de LDL)",
        y        = NULL
    ) +
    theme_minimal(base_size=13) +
    theme(legend.position="none")

---
### IV.5 — Scatter plot final

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4d** — Tracez le scatter plot $\hat{\beta}_{CAD}$ ~ $\hat{\beta}_{LDL}$ avec les trois droites de régression superposées.  
Quel SNP s'éloigne le plus de la tendance générale ? Regardez son gène — avez-vous une hypothèse ?

</div>

In [ ]:
%%R
# Droites des 3 méthodes
slope_ivw    <- ivw_est
slope_egger  <- egger_slope
int_egger    <- egger_intercept
slope_wm     <- wm_est

# Identifier les outliers potentiels
mr_data$outlier <- abs(mr_data$wald_ratio - ivw_est) > 3 * mr_data$wald_se

ggplot(mr_data, aes(x=BETA_LDL, y=BETA_CAD)) +
    geom_point(aes(size=F_stat,
                   color=outlier), alpha=0.7) +
    geom_label(data=subset(mr_data, outlier),
               aes(label=GENE), size=3, nudge_y=0.01, alpha=0.8) +
    # Droite IVW (sans intercept)
    geom_abline(intercept=0,          slope=slope_ivw,
                color="#2166ac", linewidth=1,   linetype="solid") +
    # Droite MR-Egger (avec intercept)
    geom_abline(intercept=int_egger,  slope=slope_egger,
                color="#d73027", linewidth=1,   linetype="dashed") +
    # Droite Weighted Median (sans intercept)
    geom_abline(intercept=0,          slope=slope_wm,
                color="#1a9850", linewidth=1,   linetype="dotted") +
    scale_color_manual(values=c("FALSE"="grey50","TRUE"="#e08214"),
                       labels=c("FALSE"="Normal","TRUE"="Outlier potentiel")) +
    labs(
        title    = "MR LDL-C → CAD — Scatter plot",
        subtitle = "Bleu: IVW  |  Rouge: MR-Egger  |  Vert: Weighted Median",
        x        = expression(hat(beta)[LDL]~"(effet SNP sur LDL-C)"),
        y        = expression(hat(beta)[CAD]~"(effet SNP sur CAD, log OR)"),
        color    = NULL, size = "F-stat"
    ) +
    theme_minimal(base_size=12)

---
## Synthèse

<div style="background-color:#e8f4f8; padding:12px; border-radius:6px;">

**Questions de synthèse :**

1. Résumez en 3 phrases l'évidence causale entre LDL et CAD obtenue dans ce TP.

2. Les trois méthodes (IVW, MR-Egger, Weighted Median) convergent-elles ?  
   Qu'est-ce que cela implique sur la qualité des instruments ?

3. L'intercept de MR-Egger est-il préoccupant ici ? Dans quel cas le serait-il ?

4. En quoi le cis-eQTL de *HMGCR* est-il un instrument *particulièrement* intéressant  
   pour tester l'effet des statines par MR ?

5. **Bonus** : le SNP dans le gène *LPA* a un ratio de Wald différent des autres.  
   Cherchez pourquoi biologiquement — est-ce de la pleiotropie ou un effet biologique réel ?

</div>

---

*Données eQTL simulées à partir de GTEx v8 (Liver, HMGCR, rs12916 — GTEx Consortium 2020).*  
*Données MR basées sur Willer et al. Nat Genet 2013 (LDL) et Nikpay et al. Nat Genet 2015 (CAD).*  
*Weighted Median : Bowden et al. Genet Epidemiol 2016. MR-Egger : Bowden et al. Int J Epidemiol 2015.*